In [1]:
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import py7zr
import os
import shutil
import getpass
import geoparquet_io as gpio

#### Gestion du proxy

In [5]:
http_proxy = getpass.getpass(prompt='http_proxy')

http_proxy ········


In [6]:
https_proxy = getpass.getpass(prompt='https_proxy')

https_proxy ········


In [96]:
req = "https://geoservices.ign.fr/ocsge#telechargement"

In [7]:
proxies = { "http"  : http_proxy, "https" : https_proxy}

In [179]:
page = requests.get(req, proxies=proxies)

In [180]:
soup = BeautifulSoup(page.text, 'html.parser')

In [181]:
downloads = soup.findAll('a', href=True)

Liste des départements disponibles

In [241]:
liste_ocsge = [download for download in downloads if "OCS-GE_2-0" in download['href']]
liste_ocsge = pd.DataFrame({ 'name' : [item.find('strong').text if item.find('strong') else None for item in liste_ocsge],
               'link' : [item['href'] for item in liste_ocsge] })

liste_ocsge['codgeo'] = liste_ocsge.name.str.split(" - ",expand=True)[0].str.slice(12,).str.strip()
liste_ocsge['year'] = liste_ocsge.name.str.extract(r'(\b\d{4}\b)')

liste_ocsge = liste_ocsge.loc[liste_ocsge.link.str.find('DIFF')<0,:]

liste_ocsge.loc[liste_ocsge.codgeo.isna(), 'codgeo'] = liste_ocsge.loc[liste_ocsge.codgeo.isna()].link.map(lambda x: x.split('/')[-1][26:28])
liste_ocsge.loc[liste_ocsge.year.isna(), 'year'] = liste_ocsge.loc[liste_ocsge.year.isna()].link.map(lambda x: x.split('/')[-1][29:33])

In [242]:
liste_ocsge = liste_ocsge.sort_values(by=['codgeo', 'year'], ascending=False)
liste_ocsge = liste_ocsge[['codgeo', 'year', 'link']].groupby(['codgeo']).head(1)

In [243]:
liste_ocsge = liste_ocsge[~liste_ocsge.codgeo.isin(['971', '972', '973', '974', '8S'])]

In [245]:
liste_ocsge.to_json('dep_dispo_2026_01_07.json')

### Ouverture de la liste et traitement

In [2]:
liste_ocsge = pd.read_json('dep_dispo_2026_01_07.json')

In [3]:
liste_ocsge

,codgeo,year,link
168,95,2021,https://data.geopf.fr/telechargement/download/...
165,94,2021,https://data.geopf.fr/telechargement/download/...
162,93,2021,https://data.geopf.fr/telechargement/download/...
159,92,2021,https://data.geopf.fr/telechargement/download/...
156,91,2021,https://data.geopf.fr/telechargement/download/...
...,...,...,...
288,05,2022,https://data.geopf.fr/telechargement/download/...
285,04,2021,https://data.geopf.fr/telechargement/download/...
3,03,2022,https://data.geopf.fr/telechargement/download/...
132,02,2021,https://data.geopf.fr/telechargement/download/...


Téléchargement

In [8]:
for url in liste_ocsge.link:
  with requests.get(url, stream=True, proxies=proxies) as response:
    if response.status_code == 200:
        with open(url.rsplit('/', 1)[1], 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

  search_string = "OCCUPATION_SOL"
  with py7zr.SevenZipFile(url.rsplit('/', 1)[1], mode='r') as archive:
        # Get the list of files in the archive
        all_files = archive.getnames()

        # Filter files that contain the search string
        filtered_files = [f for f in all_files if search_string in f]

        # Extract only the filtered files
        if filtered_files:
          archive.extract(targets=filtered_files, path='ocsge_temp',recursive=False)

        for file in filtered_files:
                source_file = os.path.join('ocsge_temp', file)
                destination_file = os.path.join('ocsge', os.path.basename(file))
                os.makedirs(os.path.dirname(destination_file), exist_ok=True)
                if os.path.exists(source_file):  # Only move if the file was extracted
                    os.rename(source_file, destination_file)

  ocsge = gpd.read_file('ocsge/OCCUPATION_SOL.gpkg')
  ocsge.to_parquet('ocsge/OCCUPATION_SOL.parquet')
  gpio.read('ocsge/OCCUPATION_SOL.parquet') \
    .add_bbox() \
    .sort_hilbert() \
    .write('files_ocsge/' + url.rsplit('/', 1)[1].rsplit('.', 1)[0]+".parquet", compression='ZSTD')

    
  #ocsge = gpd.read_file('ocsge/OCCUPATION_SOL.gpkg')
  #ocsge.to_parquet('files_ocsge/' + url.rsplit('/', 1)[1].rsplit('.', 1)[0]+".parquet")

  shutil.rmtree('ocsge_temp')
  shutil.rmtree('ocsge')
  os.remove(url.rsplit('/', 1)[1])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
toto = gpd.read_file('ocsge/OCCUPATION_SOL.gpkg')

In [18]:
toto

,id,code_cs,code_us,millesime,source,ossature,id_origine,code_or,geometry
0,OCSGE0000000010012067564,CS1.1.1.2,US4.1.1,2021,BDTopo,1,TRONROUT0000000003981882,OR1.2,"MULTIPOLYGON (((658379 6892064.24, 658378.41 6..."
1,OCSGE0000000010012063004,CS2.1.1.1,US1.1,2021,calcul,0,NC,NC,"MULTIPOLYGON (((664381.32 6883392.14, 664453.7..."
2,OCSGE0000000010012067614,CS1.1.1.2,US4.1.1,2021,calcul,1,None,OR2.1.1,"MULTIPOLYGON (((664594.98 6877222.9, 664589.22..."
3,OCSGE0000000010012063632,CS1.1.1.1,US235,2021,calcul,0,NC,NC,"MULTIPOLYGON (((661706.8 6877938.8, 661701.09 ..."
4,OCSGE0000000010012070973,CS1.1.1.1,US5,2021,calcul,0,NC,NC,"MULTIPOLYGON (((644098.56 6891109.94, 644096.5..."
...,...,...,...,...,...,...,...,...,...
151005,OCSGE0000000010012067581,CS1.1.1.2,US4.1.1,2021,calcul,1,None,OR2.1.1,"MULTIPOLYGON (((664543.22 6877271.3, 664542.9 ..."
151006,OCSGE0000000010012088522,CS2.2.1,US3,2021,calcul,0,NC,NC,"MULTIPOLYGON (((655911.91 6890320.29, 655910.9..."
151007,OCSGE0000000010012152980,CS2.1.1.1,US1.2,2021,calcul,0,NC,NC,"MULTIPOLYGON (((633311.07 6888535.07, 633310.4..."
151008,OCSGE0000000010012153015,CS2.2.1,US1.1,2021,calcul,0,NC,NC,"MULTIPOLYGON (((631294.25 6892951.14, 631293.0..."
